In [2]:
import h5py
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

C:\Users\GlassDesktop\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
def read_dream3d_file(file_path, dataset_name):
    """
    Reads a dataset named `dataset_name` from the 'CellFeatureData' group in a DREAM.3D file.
    Returns the data as a NumPy array.
    """
    with h5py.File(file_path, 'r') as file:
        group = file['DataContainers/ImageDataContainer/CellFeatureData']
        dataset = group[dataset_name]
        data = dataset[...]
    return data

file_IA = '../../3D/BA/BA_microstructure_cleanfilled_grainshape_misor_KAM_IPF.dream3d'
quatIA = read_dream3d_file(file_IA, 'rotated_ori')  # Expected shape: (N,4)
grainsIA = read_dream3d_file(file_IA, 'grainID')      # Expected shape: (N,) or (N,1)

if grainsIA.ndim > 1:
    grainsIA = grainsIA.flatten()

if quatIA.ndim > 1 and quatIA.shape[1] > 4:
    quatIA = quatIA[:, :4]

norms = np.linalg.norm(quatIA, axis=1, keepdims=True)
quatIA = quatIA / norms

def misorientation_angle(q1, q2):
    """
    Computes the misorientation angle (in degrees) between two unit quaternions.
    Both q1 and q2 should be arrays of shape (4,).  
    Uses: angle = 2 * arccos(|dot(q1, q2)|)
    """
    q1 = np.asarray(q1).flatten()
    q2 = np.asarray(q2).flatten()
    dot = abs(np.dot(q1, q2))
    dot = np.clip(dot, 0, 1)
    angle_rad = 2 * np.arccos(dot)
    return np.degrees(angle_rad)

cube_variants_euler = [
    (0, 0, 0),
    (0, 0, 90),
    (0, 0, 180),
    (90, 90, 0),
    (90, 90, 90),
    (90, 90, 180)
]
cube_variants_quat = R.from_euler('zxz', cube_variants_euler, degrees=True).as_quat()

def is_cube_quaternion(q, tol):
    """
    Returns True if quaternion q (shape (4,)) is within tol degrees (misorientation)
    of any cube variant quaternion.
    """
    for cq in cube_variants_quat:
        if misorientation_angle(q, cq) < tol:
            return True
    return False

tol_degs = 10  # Tolerance in degrees
cube_mask_IA = np.array([is_cube_quaternion(q, tol=tol_degs) for q in quatIA])
cube_grains_IA = np.array(grainsIA)[cube_mask_IA]

print(f"IA Alloy: Found {len(cube_grains_IA)} cube-oriented grains.")
print("Cube-oriented Grain IDs (IA):", cube_grains_IA)

pd.Series(cube_grains_IA, name="CubeOrientedGrainIDs").to_csv("CubeGrains_IA.csv", index=False)


C:\Users\GlassDesktop\AppData\Local\Temp\ipykernel_26004\2282510699.py:29: RuntimeWarning: invalid value encountered in divide
  quatIA = quatIA / norms


IA Alloy: Found 34 cube-oriented grains.
Cube-oriented Grain IDs (IA): [ 106  107  116  145  156  244  297  364  381  531  663  738 1124 1158
 1182 1279 1343 1443 1505 1704 1729 1732 1738 1739 1839 1901 1903 1934
 1992 2088 2099 2127 2136 2203]


In [6]:

file_NoIA = '../../3D/NoBA/NoBA_microstructure_cleanfilled_grainshape_misor_KAM_IPF.dream3d'
quatNoIA = read_dream3d_file(file_NoIA, 'rotated_ori')  # Expected shape: (N,4)
grainsNoIA = read_dream3d_file(file_NoIA, 'grainID')      # Expected shape: (N,) or (N,1)

if grainsNoIA.ndim > 1:
    grainsNoIA = grainsNoIA.flatten()
if quatNoIA.ndim > 1 and quatNoIA.shape[1] > 4:
    quatNoIA = quatNoIA[:, :4]

norms = np.linalg.norm(quatNoIA, axis=1, keepdims=True)
quatNoIA = quatNoIA / norms

tol_degs = 10  # Tolerance in degrees
cube_mask_NoIA = np.array([is_cube_quaternion(q, tol=tol_degs) for q in quatNoIA])
cube_grains_NoIA = np.array(grainsNoIA)[cube_mask_NoIA]

print(f"No IA Alloy: Found {len(cube_grains_NoIA)} cube-oriented grains.")
print("Cube-oriented Grain IDs (No IA):", cube_grains_NoIA)

pd.Series(cube_grains_NoIA, name="CubeOrientedGrainIDs").to_csv("CubeGrains_NoIA.csv", index=False)


C:\Users\GlassDesktop\AppData\Local\Temp\ipykernel_26528\130432836.py:15: RuntimeWarning: invalid value encountered in divide
  quatNoIA = quatNoIA / norms


No IA Alloy: Found 73 cube-oriented grains.
Cube-oriented Grain IDs (No IA): [  49   72  158  182  275  287  307  315  333  349  373  379  410  440
  458  493  506  508  518  578  598  600  615  677  754  770  782  857
  864  958  970  971  990 1007 1008 1063 1153 1196 1294 1321 1373 1392
 1457 1494 1545 1556 1592 1631 1654 1664 1674 1680 1802 1908 1916 1961
 1962 1987 2001 2003 2009 2034 2066 2081 2101 2102 2140 2217 2253 2271
 2342 2378 2410]
